In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import random
import os

# Set seeds for reproducibility
def set_seed(seed=42):
    np.random.seed(seed)
    random.seed(seed)
    tf.random.set_seed(seed)

# Define the U-Net model
def unet_model(input_shape=(256, 256, 1)):
    inputs = keras.Input(shape=input_shape)

    # Contracting Path
    c1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    c1 = layers.BatchNormalization()(c1)
    c1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c1)
    c1 = layers.BatchNormalization()(c1)
    p1 = layers.MaxPooling2D((2, 2))(c1)

    c2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
    c2 = layers.BatchNormalization()(c2)
    c2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c2)
    c2 = layers.BatchNormalization()(c2)
    p2 = layers.MaxPooling2D((2, 2))(c2)

    # Bottleneck
    c3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(p2)
    c3 = layers.BatchNormalization()(c3)
    c3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(c3)
    c3 = layers.BatchNormalization()(c3)

    # Expanding Path
    u1 = layers.UpSampling2D((2, 2))(c3)
    u1 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(u1)
    u1 = layers.BatchNormalization()(u1)
    u1 = layers.Concatenate()([u1, c2])

    u2 = layers.UpSampling2D((2, 2))(u1)
    u2 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(u2)
    u2 = layers.BatchNormalization()(u2)
    u2 = layers.Concatenate()([u2, c1])

    outputs = layers.Conv2D(1, (1, 1), activation='sigmoid')(u2)

    model = keras.Model(inputs, outputs)
    return model

# Dice Similarity Coefficient (DSC) Loss
def dice_coefficient(y_true, y_pred, smooth=1):
    y_true_f = tf.keras.backend.flatten(tf.cast(y_true, tf.float32))  # Convert to float32
    y_pred_f = tf.keras.backend.flatten(y_pred)
    intersection = tf.keras.backend.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (tf.keras.backend.sum(y_true_f) + tf.keras.backend.sum(y_pred_f) + smooth)


# Compile the model
def compile_model():
    model = unet_model()
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[dice_coefficient])
    return model

# Perform Monte Carlo training runs
def monte_carlo_training(data, labels, runs=10):
    results = []
    for i in range(runs):
        print(f"Monte Carlo Run {i+1}/{runs}")
        set_seed(i)
        model = compile_model()
        history = model.fit(data, labels, epochs=5, batch_size=8, validation_split=0.2, verbose=1)
        results.append(history.history['val_dice_coefficient'][-1])
    return results

# Placeholder for data loading
def load_data():
    # This should be replaced with actual DCE-MRI dataset loading
    data = np.random.rand(100, 256, 256, 1)  # Dummy data
    labels = np.random.randint(0, 2, (100, 256, 256, 1))  # Dummy labels
    return data, labels

# Main execution
data, labels = load_data()
monte_carlo_results = monte_carlo_training(data, labels, runs=10)
print("Monte Carlo DSC Scores:", monte_carlo_results)


Monte Carlo Run 1/10
Epoch 1/5


KeyboardInterrupt: 

In [5]:
import kagglehub
import numpy as np
import tensorflow as tf
import os
import cv2
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers

# Set seeds for reproducibility
def set_seed(seed=42):
    np.random.seed(seed)
    tf.random.set_seed(seed)


#/////////////////
import os

dataset_path = kagglehub.dataset_download("masoudnickparvar/brain-tumor-mri-dataset")
print("Dataset downloaded to:", dataset_path)

# List the contents of the dataset directory
print("Files and directories in dataset path:")
print(os.listdir(dataset_path))

# List contents inside 'Training' and 'Testing' directories
training_path = os.path.join(dataset_path, "Training")
testing_path = os.path.join(dataset_path, "Testing")

if os.path.exists(training_path):
    print("Contents of Training directory:", os.listdir(training_path))
else:
    print("Training directory not found!")

if os.path.exists(testing_path):
    print("Contents of Testing directory:", os.listdir(testing_path))
else:
    print("Testing directory not found!")

# Load dataset
def load_data():
    images, masks = [], []
        dataset_path = kagglehub.dataset_download("masoudnickparvar/brain-tumor-mri-dataset")
print("Dataset downloaded to:", dataset_path)

# List all files & folders
for root, dirs, files in os.walk(dataset_path):
    print(f"\n📂 Directory: {root}")
    for d in dirs:
        print(f"  📁 {d}")
    for f in files:
        print(f"  📄 {f}")
        if os.path.exists(mask_path):
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # Load as grayscale
            mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

            img = cv2.resize(img, (256, 256)) / 255.0  # Normalize image
            mask = cv2.resize(mask, (256, 256)) / 255.0  # Normalize mask

            images.append(img)
            masks.append(mask)

    images = np.array(images).reshape(-1, 256, 256, 1)
    masks = np.array(masks).reshape(-1, 256, 256, 1)

    # Split into train and validation sets
    train_images, val_images, train_masks, val_masks = train_test_split(images, masks, test_size=0.2, random_state=42)

    return (train_images, train_masks), (val_images, val_masks)

#//////////////
dataset_path = kagglehub.dataset_download("masoudnickparvar/brain-tumor-mri-dataset")
print("Dataset downloaded to:", dataset_path)

# List all files & folders
for root, dirs, files in os.walk(dataset_path):
    print(f"\n📂 Directory: {root}")
    for d in dirs:
        print(f"  📁 {d}")
    for f in files:
        print(f"  📄 {f}")


# Define U-Net model
def unet_model(input_shape=(256, 256, 1)):
    inputs = keras.Input(shape=input_shape)

    # Contracting Path
    c1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    c1 = layers.BatchNormalization()(c1)
    c1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c1)
    c1 = layers.BatchNormalization()(c1)
    p1 = layers.MaxPooling2D((2, 2))(c1)

    c2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
    c2 = layers.BatchNormalization()(c2)
    c2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c2)
    c2 = layers.BatchNormalization()(c2)
    p2 = layers.MaxPooling2D((2, 2))(c2)

    # Bottleneck
    c3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(p2)
    c3 = layers.BatchNormalization()(c3)
    c3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(c3)
    c3 = layers.BatchNormalization()(c3)

    # Expanding Path
    u1 = layers.UpSampling2D((2, 2))(c3)
    u1 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(u1)
    u1 = layers.BatchNormalization()(u1)
    u1 = layers.Concatenate()([u1, c2])

    u2 = layers.UpSampling2D((2, 2))(u1)
    u2 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(u2)
    u2 = layers.BatchNormalization()(u2)
    u2 = layers.Concatenate()([u2, c1])

    outputs = layers.Conv2D(1, (1, 1), activation='sigmoid')(u2)

    model = keras.Model(inputs, outputs)
    return model

# Dice Similarity Coefficient (DSC) Loss
def dice_coefficient(y_true, y_pred, smooth=1):
    y_true_f = tf.keras.backend.flatten(tf.cast(y_true, tf.float32))
    y_pred_f = tf.keras.backend.flatten(y_pred)
    intersection = tf.keras.backend.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (tf.keras.backend.sum(y_true_f) + tf.keras.backend.sum(y_pred_f) + smooth)

# Compile the model
def compile_model():
    model = unet_model()
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[dice_coefficient])
    return model

# Monte Carlo Training
def monte_carlo_training(train_data, train_labels, val_data, val_labels, runs=10):
    results = []
    for i in range(runs):
        print(f"Monte Carlo Run {i+1}/{runs}")
        set_seed(i)
        model = compile_model()
        history = model.fit(
            train_data, train_labels,
            epochs=5,
            batch_size=8,
            validation_data=(val_data, val_labels),
            verbose=1
        )
        results.append(history.history['val_dice_coefficient'][-1])
    return results

# Load dataset
(train_data, train_labels), (val_data, val_labels) = load_data()

# Perform Monte Carlo training
monte_carlo_results = monte_carlo_training(train_data, train_labels, val_data, val_labels, runs=10)
print("Monte Carlo DSC Scores:", monte_carlo_results)


Dataset downloaded to: /root/.cache/kagglehub/datasets/masoudnickparvar/brain-tumor-mri-dataset/versions/1
Files and directories in dataset path:
['Testing', 'Training']
Contents of Training directory: ['glioma', 'notumor', 'pituitary', 'meningioma']
Contents of Testing directory: ['glioma', 'notumor', 'pituitary', 'meningioma']


FileNotFoundError: [Errno 2] No such file or directory: '/root/.cache/kagglehub/datasets/masoudnickparvar/brain-tumor-mri-dataset/versions/1/Training/images'